In [1]:

import jsonlines
import ndjson
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pecos

ModuleNotFoundError: No module named 'jsonlines'

# 1 Import the libriaries

Using Anaconda or other python manager to install the above libraries. On anaconda use cmd: conda install ....python package ..

# 2 Set the array variables for OpenAQ dataset 

In [ ]:
data = []

# 3 Import the OpenAQ Dataset

### 3 It uses the OpenAQ Dataset of a day in 2018 

### 3 The default use OpenAQ_1.ndjson (a newline delimited format)

### 3 Choose another one of the OpenAQ datasets in ndjson by changing the address of open('......')

In [ ]:
with jsonlines.open('OpenAQ_1.ndjson') as reader:
    for obj in reader:
        #print(obj)
        obj1 = pd.Series(obj)
        #pd.read_csv(obj)
        data.append(obj1)


# 4 Import OpenAQ dataset to Pandas DataFrame

In [ ]:
Dataset = pd.DataFrame(data)

# 5 Define the ploting method

In [ ]:
def plot_df(df, x, y, title="", xlabel='Date', ylabel='Value', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

# 6 Choose and print some variables from Pandas Dataframe

In [ ]:
print(Dataset[['date','value','parameter','location']])

print(Dataset.dtypes)

# 7 Split the 'date' variable to two variables because it has utc and local dates in one variable

In [ ]:
#How to split column into two columns
#https://www.geeksforgeeks.org/split-a-text-column-into-two-columns-in-pandas-dataframe/

Dataset[['Dateutc','Datelocal']] = Dataset.date.apply(lambda x: pd.Series(str(x).split(",")))

Dataset_split = Dataset.Dateutc.apply(lambda x: pd.Series(str(x).split(":")))

#print(Dataset_split)


# 8 Convert new 'utc' (Date in utc) variable to date format for Pandas

In [ ]:
Dataset['utc'] = pd.to_datetime(Dataset_split[1])

#print(Dataset.dtypes)

#print(Dataset)

pd.to_datetime(Dataset['utc'])

# 9 Copy OpenAQ dataset 

In [ ]:
pd.to_datetime(Dataset2['utc'])
#Dataset2.set_index('utc')

# 10 Copy OpenAQ Dataset variable 'location' 

In [ ]:
Location_Subset = Dataset[['location']].copy()

# 11 Sort descending and remove duplicate to get all unique station's 'location'

In [ ]:
Location_Subset.sort_values('location', ascending=False)

Location_Subset = Location_Subset.drop_duplicates(subset='location', keep='first')

# 12 Set the OpenAQ dataset variable 'utc' (The date in utc) to the index

In [ ]:
Dataset2.index = Dataset2['utc']

# 13 The setting the array variables results 

In [ ]:
Test_results_location_subset = []

i = 0

pm2 = []


# 14 The iteration over every station 'location' in the Location_Subset array 

# 15 Iterate over every AQ variable i.e. No2, So2, PM10, PM25 that the location has 

## 15 It only iterates over the AQ variables that are measured at the choosen location

In [ ]:

for LocationId in Location_Subset['location']:
 
    print(LocationId)   
  
    Test_results_location_subset_parameter = []
    
    # 14 i Append the location being searched to array variable 
    
    Test_results_location_subset_parameter.append(LocationId)
    
    # 14 ii Filter the OpenAQ Dataset for the location being searched for   
    
    Dataset3 = Dataset2[Dataset2['location']==LocationId]
    
    # 14 iii Format the utc variable to Date format of Pandas
    
    pd.to_datetime(Dataset3['utc'])
    
    #14 iv Copy all the AQ variables of searched for station
    
    Test_results_location_subset_parameter_subset = Dataset3[['parameter']].copy() 
         
    #print(Test_results_location_subset_parameter_subset)
    
    #14 v Sort and remove duplicates getting unique AQ Variables
    
    Test_results_location_subset_parameter_subset.sort_values('parameter', ascending=False)
    
    Test_results_location_subset_parameter_subset = Test_results_location_subset_parameter_subset.drop_duplicates(subset='parameter', keep='first')

    #15 Iterate over the AQ Variables 
    
    for parameter in Test_results_location_subset_parameter_subset['parameter']:
        
        print(parameter)
        
        #15 i Append AQ variable to array
        
        Test_results_parameter = []
        
        Test_results_parameter.append(parameter)
        
        # 15 ii Filter the OpenAQ dataset for AQ Variable 
        
        Dataset4 = Dataset3[Dataset3['parameter']==parameter]      

        # 15 iv The get variable of Date from utc variable
        
        Dataset4['year'] = Dataset4['utc'].dt.year
        
        Dataset4['month'] = Dataset4['utc'].dt.month

        Dataset4['day'] = Dataset4['utc'].dt.day
        
        pd.to_datetime(Dataset4['utc'])

        Dataset5 = Dataset4;

        #15 v The Pecos package to find errors in the OpenAQ dataset choosen on criteria
        
        # 15 v Initialize logger
        pecos.logger.initialize()

        # 15 v Create a Pecos PerformanceMonitoring data object
        pm1 = pecos.monitoring.PerformanceMonitoring()
        
        pm1.add_dataframe(Dataset4)
        
        # 15 vi Check for missing values
    
        pm1.check_missing()
        
        # 15 vii Check for corrupt data values
        
        ###### Change value -999.00000 to other value that are not accepted 
        
        pm1.check_corrupt([-999.000000],'value') 

# Add a composite signal which compares measurements to a model
#wave_model = np.array(np.sin(10*clock_time/86400))
#wave_measurments = pm.df[pm.trans['Wave']]
#wave_error = np.abs(wave_measurments.subtract(wave_model,axis=0))
#wave_error.columns=['Wave Error C', 'Wave Error D']
#pm.add_dataframe(wave_error)
#pm.add_translation_dictionary({'Wave Error': ['Wave Error C', 'Wave Error D']})

        # 15 viii Check data for expected ranges
        
        #### CHOOSE criteria ### Change [0,10000000] to selected lower and high bound ###
        
        pm1.check_range([0, 10000000], 'value')

        # 15 iix Check that the Date utc value is accurate i.e. expected measurements

        pm1.check_range([2017, 2018], 'year')
        pm1.check_corrupt([2,3,4,5,6,7,8,9,10,11],'month')
        pm1.check_corrupt([2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30],'day')
        
        # 15 ix Append results to Array

        print(pm1.test_results)
        
        Test_results_parameter.append(len(pm1.test_results))
        
        Test_results_parameter.append(pm1.test_results)
        Test_results_parameter.append(Dataset5)
        
        Test_results_location_subset_parameter.append(Test_results_parameter)
        
        
        
        #i = i + 1
       

    Test_results_location_subset.append(Test_results_location_subset_parameter)
    


# 16 Print Results

In [ ]:
print(Test_results_location_subset)

# 17 Output results to Excel

In [ ]:
for Test_results_location in Test_results_location_subset:
    
    for Test_Parameter in Test_results_location[1:]:
        
        pecos.io.write_test_results(Test_Parameter)